In [14]:
import pandas as pd
import pandas as pd
import numpy as np
import openpyxl

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

tf.config.threading.set_intra_op_parallelism_threads(0)
tf.config.threading.set_inter_op_parallelism_threads(12)

In [56]:
df = pd.read_csv('/home/joao/workspace/lichess/datasets_csv/ml_ready.csv', index_col=0)
df = df.drop(columns=['Board'])
df = df[['MoveNumber', 'WhiteElo', 'BlackElo', 'Result', 'K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p']]
# df = df[['MoveNumber', 'Result', 'K', 'Q', 'R', 'B', 'N', 'P', 'k', 'q', 'r', 'b', 'n', 'p']]
# df = df[['WhiteElo', 'BlackElo', 'Result']]
print(df)
df['Result'] = df['Result'].apply(lambda x: '0' if x=='1-0' else '1' if x=='0-1' else '2')
df['Result'].value_counts()
df['Result'] = df['Result'].astype(int)

# remove draws for now
df = df[df['Result'] != 2]

                                 MoveNumber  WhiteElo  BlackElo Result  K  Q  \
Id                                                                             
https://lichess.org/9xFY5fim-0            0       956       880    0-1  1  1   
https://lichess.org/9xFY5fim-1            1       956       880    0-1  1  1   
https://lichess.org/9xFY5fim-2            2       956       880    0-1  1  1   
https://lichess.org/9xFY5fim-3            3       956       880    0-1  1  1   
https://lichess.org/9xFY5fim-4            4       956       880    0-1  1  1   
...                                     ...       ...       ...    ... .. ..   
https://lichess.org/42ssxwWO-30          30       991       897    1-0  1  1   
https://lichess.org/42ssxwWO-31          31       991       897    1-0  1  1   
https://lichess.org/42ssxwWO-32          32       991       897    1-0  1  1   
https://lichess.org/42ssxwWO-33          33       991       897    1-0  1  1   
https://lichess.org/42ssxwWO-34         

## Train-test on the last 20%

In [58]:

# Split the data into features and labels
X = df.drop(['Result', 'MoveNumber'], axis=1).values
y = df['Result'].values


# Split the data into training and validation sets
train_size = int(len(X) * 0.8)
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]



# Define the model architecture
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X.shape[1]))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model on the training data
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10240, verbose=1)

# Evaluate the model on the validation data
loss, acc = model.evaluate(X_val, y_val, verbose=1)
print('Validation accuracy:', acc)
print('Validation loss:', loss)

Epoch 1/100
16/16 [==============================] - 1s 23ms/step - loss: 1.0499 - accuracy: 0.5013 - val_loss: 0.6938 - val_accuracy: 0.5197
Epoch 2/100
16/16 [==============================] - 0s 16ms/step - loss: 0.6977 - accuracy: 0.5039 - val_loss: 0.6931 - val_accuracy: 0.5199
Epoch 3/100
16/16 [==============================] - 0s 16ms/step - loss: 0.6916 - accuracy: 0.5158 - val_loss: 0.6894 - val_accuracy: 0.5199
Epoch 4/100
16/16 [==============================] - 0s 15ms/step - loss: 0.6888 - accuracy: 0.5447 - val_loss: 0.6884 - val_accuracy: 0.5827
Epoch 5/100
16/16 [==============================] - 0s 15ms/step - loss: 0.6865 - accuracy: 0.5770 - val_loss: 0.6864 - val_accuracy: 0.6038
Epoch 6/100
16/16 [==============================] - 0s 15ms/step - loss: 0.6850 - accuracy: 0.5738 - val_loss: 0.6844 - val_accuracy: 0.5901
Epoch 7/100
16/16 [==============================] - 0s 15ms/step - loss: 0.6830 - accuracy: 0.5953 - val_loss: 0.6835 - val_accuracy: 0.5823
Epoch 

In [59]:
# Rebuild a dataframe to check predictions
pred_probs = model.predict(X_val)
df2 = df.iloc[train_size:].copy()
df2['Prediction_Probs'] = pred_probs
df2['Predictions'] = (pred_probs >= 0.5).astype(int)


1265/1265 [==============================] - 1s 975us/step


In [63]:
df3 = df2[(df2['MoveNumber'] >= 0)]

sum(df3['Predictions'] == df3['Result'])/len(df3['Result'])
# df2.to_excel('tmp.xlsx')

0.751791981808295

In [42]:
df3[df3['Predictions'] != df3['Result']]

,MoveNumber,Result,K,Q,R,B,N,P,k,q,r,b,n,p,Prediction_Probs,Predictions
Id,,,,,,,,,,,,,,,,
https://lichess.org/Ipr5q4Nl-2,2,0,1,1,2,2,2,8,1,1,2,2,2,8,0.503885,1
https://lichess.org/fj2joVWk-2,2,0,1,1,2,2,2,8,1,1,2,2,2,8,0.503885,1
https://lichess.org/OWMKN5yW-2,2,0,1,1,2,2,2,8,1,1,2,2,2,8,0.503885,1
https://lichess.org/yEf4DM72-2,2,0,1,1,2,2,2,8,1,1,2,2,2,8,0.503885,1
https://lichess.org/jQk1bhea-2,2,0,1,1,2,2,2,8,1,1,2,2,2,8,0.503885,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
https://lichess.org/XR8H89hs-2,2,0,1,1,2,2,2,8,1,1,2,2,2,8,0.503885,1
https://lichess.org/XklIeaG4-2,2,0,1,1,2,2,2,8,1,1,2,2,2,8,0.503885,1
https://lichess.org/0jGhgiU7-2,2,0,1,1,2,2,2,8,1,1,2,2,2,8,0.503885,1


## CROSS-VALIDATION

In [21]:
# Split the data into features and labels
X = df.drop('Result', axis=1).values
y = df['Result'].values
y = to_categorical(y)

# Define the number of folds for cross-validation
num_folds = 5

# Define the KFold object with shuffle=True
kfold = KFold(n_splits=num_folds, shuffle=True)

# Define the model architecture
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=X.shape[1]))
model.add(Dense(32, activation='relu', input_dim=32))
model.add(Dense(32, activation='relu', input_dim=32))
model.add(Dense(32, activation='relu', input_dim=32))
model.add(Dense(32, activation='relu', input_dim=32))
# model.add(Dense(1, activation='sigmoid'))
model.add(Dense(2, activation='softmax'))

# Compile the model
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Loop over the folds
fold_acc = []
fold_loss = []
for fold, (train_idx, val_idx) in enumerate(kfold.split(X, y)):
    print('Fold:', fold+1)
    # Get the training and validation data for this fold
    X_train, y_train = X[train_idx], y[train_idx]
    X_val, y_val = X[val_idx], y[val_idx]
    
    # Fit the model on the training data for this fold
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=10240, verbose=0)
    
    # Evaluate the model on the validation data for this fold
    loss, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Validation accuracy:', acc)
    print('Validation loss:', loss)
    fold_acc.append(acc)
    fold_loss.append(loss)

# Print the average accuracy and loss over all folds
print('Average accuracy:', np.mean(fold_acc))
print('Average loss:', np.mean(fold_loss))


Fold: 1
Validation accuracy: 0.8117059469223022
Validation loss: 0.32533878087997437
Fold: 2
Validation accuracy: 0.8135102987289429
Validation loss: 0.3130733072757721
Fold: 3
Validation accuracy: 0.8131642937660217
Validation loss: 0.30507153272628784
Fold: 4
Validation accuracy: 0.8249993920326233
Validation loss: 0.2928656339645386
Fold: 5
Validation accuracy: 0.8226264715194702
Validation loss: 0.2933903634548187
Average accuracy: 0.8172012805938721
Average loss: 0.30594792366027834
